# WaPOR v3 Data Collection and Preparation
This notebook downloads WaPOR v3 data and prepares it for water accounting analysis.

**Key updates in v3:**
- No API token required
- Direct COG-based downloads via GDAL
- Faster and more reliable data access

## 1. Import Functions and Setup Paths

In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import calendar
from osgeo import gdal
import osr

# Add WAPORWA modules to path
# modules_path = r'D:\Papers and Journals\WAPORWA\modules'  # Change to your modules path
# sys.path.insert(0, modules_path)

# Import WA modules
import WA
from WA.pickle_basin import pickle_in, pickle_out

print("✓ Modules imported successfully")

✓ Modules imported successfully


In [3]:
import sys
sys.path.insert(0, r"D:\Papers and Journals\WAPORWA\modules")

import WaPOR
WaPOR.list_available_mapsets()

print(f"✓ WaPOR module location: {WaPOR.__file__}")

WaPOR v3 module loaded
No API token required - uses public COG files
Use list_available_mapsets() to see all available datasets

Available WaPOR v3 Mapsets:
--------------------------------------------------------------------------------
L1-AETI-A            - Actual EvapoTranspiration and Interception (Global - Annual - 300m)
L1-AETI-D            - Actual EvapoTranspiration and Interception (Global - Dekadal - 300m)
L1-AETI-M            - Actual EvapoTranspiration and Interception (Global - Monthly - 300m)
L1-E-A               - Evaporation (Global - Annual - 300m)
L1-E-D               - Evaporation (Global - Dekadal - 300m)
L1-GBWP-A            - Gross biomass water productivity (Annual - 300m)
L1-I-A               - Interception (Global - Annual - 300m)
L1-I-D               - Interception (Global - Dekadal - 300m)
L1-NBWP-A            - Net biomass water productivity (Annual - 300m)
L1-NPP-D             - Net Primary Production (Global - Dekadal - 300m)
L1-NPP-M             - Net Pr

## 2. Define Basin Extent and Parameters

In [4]:
# =============================================================================
# CONFIGURATION - Update these paths for your basin
# =============================================================================

# Basin shapefile path
BASIN_SHP = r"D:\Papers and Journals\WAPORWA\data\Litani\BasinLitani.shp"

# Output folder for WaPOR data
INPUT_FOLDER = r"D:\Papers and Journals\WAPORWA\data\Litani\Input"

# Main directory for processed data
MAIN_DIR = r"D:\Papers and Journals\WAPORWA\data\Litani\Main"

# Global datasets
GLOBAL_GRAND_RESERVOIR = r"D:\Papers and Journals\WAPORWA\data\Global\GRanD\GRanD_reservoirs_v1_1_fixed.shp"
WDPA_SHAPEFILE = r"D:\Papers and Journals\WAPORWA\data\Global\WDPA\WDPA_CatIandII_17countries.shp"

# Date range for data download
START_DATE = '2020-01-01'
END_DATE = '2020-03-31'

# WaPOR data level (1 = continental 300m, 2 = national 100m)
WAPOR_LEVEL = 2

# Create output directories
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(MAIN_DIR, exist_ok=True)

print(f"✓ Configuration loaded")
print(f"  Basin: {os.path.basename(BASIN_SHP)}")
print(f"  Date range: {START_DATE} to {END_DATE}")
print(f"  WaPOR level: {WAPOR_LEVEL}")

✓ Configuration loaded
  Basin: BasinLitani.shp
  Date range: 2020-01-01 to 2020-03-31
  WaPOR level: 2


## 3. Extract Basin Bounding Box

In [5]:
# Load basin shapefile and extract bounding box
import shapefile

shape = shapefile.Reader(BASIN_SHP)
xmin, ymin, xmax, ymax = shape.bbox

print("Basin Bounding Box:")
print(f"  Longitude: {xmin:.6f} to {xmax:.6f}")
print(f"  Latitude:  {ymin:.6f} to {ymax:.6f}")

# Define lat/lon limits with buffer for different datasets
# Larger buffer for coarser resolution data (L1)
latlim_L1 = [ymin - 0.25, ymax + 0.25]
lonlim_L1 = [xmin - 0.25, xmax + 0.25]

# Exact bbox for fine resolution data (L2)
latlim_L2 = [ymin, ymax]
lonlim_L2 = [xmin, xmax]

print(f"\n✓ Bounding box extracted successfully")

Basin Bounding Box:
  Longitude: 35.229167 to 36.400000
  Latitude:  33.100000 to 34.050000

✓ Bounding box extracted successfully


### 4.3 Monthly Reference Evapotranspiration (L1/L2-RET-M)

In [6]:
import WaPOR.RET_monthly as RETv3
latlim_L1 = [ymin - 0.25, ymax + 0.25]
lonlim_L1 = [xmin - 0.25, xmax + 0.25]

RETv3.main(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L1,
    lonlim=lonlim_L1,
    level=1,   # 1 or 2
    version=3,
    Waitbar=1
)



IndentationError: unindent does not match any outer indentation level (RET_monthly.py, line 54)

In [8]:
urllib.request.urlretrieve(url, tmp_raw)


NameError: name 'urllib' is not defined

## 4. Download WaPOR v3 Data

This section downloads all required WaPOR datasets using the new v3 API.
**Note:** No API token is required for WaPOR v3!

### 4.1 Daily Precipitation (L1-PCP-E)

In [ ]:
print("="*80)
print("DOWNLOADING DAILY PRECIPITATION (L1-PCP-E)")
print("="*80)

WaPOR.PCP_daily(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L1,
    lonlim=lonlim_L1,
    version=3,  # Always uses v3
    Waitbar=1
)

print("\n✓ Daily precipitation download complete")

### 4.2 Monthly Precipitation (L1-PCP-M)

In [7]:
print("="*80)
print("DOWNLOADING MONTHLY PRECIPITATION (L1-PCP-M)")
print("="*80)

WaPOR.PCP_monthly(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L1,
    lonlim=lonlim_L1,
    version=3,
    Waitbar=1
)

print("\n✓ Monthly precipitation download complete")

DOWNLOADING MONTHLY PRECIPITATION (L1-PCP-M)

Date range: 2020-01-01 to 2020-03-31
Bounding box: lat [32.8499999999998, 34.29999999999972], lon [34.97916666666592, 36.649999999999224]
Found 3 rasters to download
Using scale factor: 0.1
[1/3] Downloading: L1-PCP-M.2020-01.tif
[2/3] Downloading: L1-PCP-M.2020-02.tif
[3/3] Downloading: L1-PCP-M.2020-03.tif
Finished downloading L1-PCP-M

✓ Monthly precipitation download complete


### 4.4 Monthly Actual Evapotranspiration (L1/L2-AETI-M)

In [9]:
import os
import gdal

# Windows-safe removal for GDAL temp files
def safe_remove(path):
    try:
        gdal.Unlink(path)    # GDAL cleanup
    except:
        pass
    try:
        os.remove(path)      # Normal cleanup
    except PermissionError:
        pass

# Patch remove globally
os.remove = safe_remove


In [7]:
import WaPOR.PCP_yearly as PCPY  # if placed inside WaPOR/modules

PCPY.main(
    Dir=INPUT_FOLDER,
    Startdate="2019-01-01",
    Enddate="2020-01-31",
    latlim=latlim_L1,
    lonlim=lonlim_L1,
    version=3,
    Waitbar=1
)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\Papers and Journals\\WAPORWA\\data\\Litani\\Input\\L1-PCP-A\\_raw_2019.tif'

In [9]:
# Check if interception data exists
import requests
from WaPOR.waporv3_api import base_url, collect_responses

mapset_url = f"{base_url}/L2-I-D/rasters"
rasters = collect_responses(mapset_url, info=["code", "downloadUrl"])

# Test first URL
if rasters:
    code, url = rasters[0]
    print(f"Testing URL: {url}")
    response = requests.head(url, timeout=30)
    print(f"Status: {response.status_code}")
    print(f"Content-Length: {response.headers.get('Content-Length', 'Unknown')}")

Testing URL: https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L2-I-D/WAPOR-3.L2-I-D.2018-01-D1.tif
Status: 200
Content-Length: 351983828


In [6]:
# Test the two-step approach manually
import requests
from osgeo import gdal
import os

url = "https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L2-I-D/WAPOR-3.L2-I-D.2018-01-D1.tif"
test_dir = r"D:\test"
os.makedirs(test_dir, exist_ok=True)

# Step 1: Download
download_path = os.path.join(test_dir, "full_download.tif")
print("Downloading...")
response = requests.get(url, stream=True)
with open(download_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)
print(f"✓ Downloaded: {os.path.getsize(download_path)} bytes")

# Step 2: Crop
bbox = [35.2, 33.1, 36.4, 34.0]  # Your basin bbox
output_path = os.path.join(test_dir, "cropped.tif")
print("Cropping...")
ds = gdal.Warp(output_path, download_path, 
               options=gdal.WarpOptions(outputBounds=bbox, dstNodata=-9999))
if ds:
    print("✓ Cropped successfully")
    ds = None
else:
    print("✗ Crop failed")

Downloading...
✓ Downloaded: 351983828 bytes
Cropping...


SystemError: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error

In [9]:
# file: tools/check_bigtiff_support.py
"""
Assert BigTIFF read support before attempting Warp. Why: avoids opaque NULL errors.
"""
from osgeo import gdal

gdal.UseExceptions()

def assert_bigtiff_read_support():
    # If this raises on your file, your GDAL/libtiff lacks BigTIFF support.
    try:
        ds = gdal.Open(r"D:\test\full_download.tif", gdal.GA_ReadOnly)
    except RuntimeError as e:
        msg = str(e)
        if "BigTIFF" in msg or "not supported" in msg:
            raise SystemExit(
                "Your GDAL build lacks BigTIFF support. Install GDAL from conda-forge or OSGeo4W (64-bit)."
            )
        raise
    finally:
        try:
            ds = None
        except Exception:
            pass

if __name__ == "__main__":
    assert_bigtiff_read_support()
    print("OK: BigTIFF supported, proceed to gdal.Warp.")


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Zuhail\AppData\Local\Temp\ipykernel_7240\1170236020.py", line 12, in assert_bigtiff_read_support
    ds = gdal.Open(r"D:\test\full_download.tif", gdal.GA_ReadOnly)
  File "C:\Users\Zuhail\anaconda3\envs\waporwa\lib\site-packages\osgeo\gdal.py", line 3087, in Open
    return _gdal.Open(*args)
RuntimeError: This is a BigTIFF file.  BigTIFF is not supported by this version of GDAL and libtiff.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Zuhail\anaconda3\envs\waporwa\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Zuhail\AppData\Local\Temp\ipykernel_7240\1170236020.py", line 27, in <module>
    assert_bigtiff_read_support()
  File "C:\Users\Zuhail\AppData\Local\Temp\ipykernel_7240\1170236020.py", line 17, in assert_bigtiff_read_support
    "Your GDAL build lacks Bi

TypeError: object of type 'NoneType' has no len()

In [8]:
gdalinfo --format GTiff

SyntaxError: invalid syntax (158106904.py, line 1)

In [28]:
print("="*80)
print(f"DOWNLOADING MONTHLY ACTUAL ET (L{WAPOR_LEVEL}-AETI-M)")
print("="*80)

WaPOR.AET_monthly(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L2 if WAPOR_LEVEL == 2 else latlim_L1,
    lonlim=lonlim_L2 if WAPOR_LEVEL == 2 else lonlim_L1,
    level=1,
    version=3,
    Waitbar=1
)

print("\n✓ Monthly actual ET download complete")

DOWNLOADING MONTHLY ACTUAL ET (L2-AETI-M)

Date range: 2020-01-01 to 2020-03-31
Bounding box: lat [33.0999999999998, 34.04999999999972], lon [35.22916666666592, 36.399999999999224]
Found 3 rasters to download
Using scale factor: 0.1
[1/3] Downloading: L1-AETI-M.2020-01.tif
ERROR downloading L1-AETI-M.2020-01.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[2/3] Downloading: L1-AETI-M.2020-02.tif
ERROR downloading L1-AETI-M.2020-02.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[3/3] Downloading: L1-AETI-M.2020-03.tif
ERROR downloading L1-AETI-M.2020-03.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
Finished downloading L1-AETI-M

✓ Monthly actual ET download complete


### 4.5 Dekadal Interception (L1/L2-I-D)

In [8]:
print("="*80)
print(f"DOWNLOADING DEKADAL INTERCEPTION (L{WAPOR_LEVEL}-I-D)")
print("="*80)

WaPOR.I_dekadal(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L2 if WAPOR_LEVEL == 2 else latlim_L1,
    lonlim=lonlim_L2 if WAPOR_LEVEL == 2 else lonlim_L1,
    level=WAPOR_LEVEL,
    version=3,
    Waitbar=1
)

print("\n✓ Dekadal interception download complete")

DOWNLOADING DEKADAL INTERCEPTION (L2-I-D)

Date range: 2020-01-01 to 2020-03-31
Bounding box: lat [33.0999999999998, 34.04999999999972], lon [35.22916666666592, 36.399999999999224]
Found 9 rasters to download
Using scale factor: 0.1
[1/9] Downloading: L2-I-D.2020-01-D1.tif
ERROR downloading L2-I-D.2020-01-D1.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[2/9] Downloading: L2-I-D.2020-01-D2.tif
ERROR downloading L2-I-D.2020-01-D2.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[3/9] Downloading: L2-I-D.2020-01-D3.tif
ERROR downloading L2-I-D.2020-01-D3.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[4/9] Downloading: L2-I-D.2020-02-D1.tif
ERROR downloading L2-I-D.2020-02-D1.tif: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error
[5/9] Downloading: L2-I-D.2020-02-D2.tif
ERROR downloading L2-I-D.2020-02-D2.tif: <built-in function wr

### 4.6 Annual Land Cover Classification (L1/L2-LCC-A)

In [ ]:
print("="*80)
print(f"DOWNLOADING ANNUAL LAND COVER (L{WAPOR_LEVEL}-LCC-A)")
print("="*80)

WaPOR.LCC_yearly(
    Dir=INPUT_FOLDER,
    Startdate=START_DATE,
    Enddate=END_DATE,
    latlim=latlim_L2 if WAPOR_LEVEL == 2 else latlim_L1,
    lonlim=lonlim_L2 if WAPOR_LEVEL == 2 else lonlim_L1,
    level=WAPOR_LEVEL,
    version=3,
    Waitbar=1
)

print("\n✓ Annual land cover download complete")

### 4.7 Download Summary

In [ ]:
print("\n" + "="*80)
print("DOWNLOAD SUMMARY")
print("="*80)

# Check downloaded data
mapsets = [
    'L1-PCP-E',
    'L1-PCP-M',
    f'L{WAPOR_LEVEL}-RET-M',
    f'L{WAPOR_LEVEL}-AETI-M',
    f'L{WAPOR_LEVEL}-I-D',
    f'L{WAPOR_LEVEL}-LCC-A'
]

for mapset in mapsets:
    mapset_dir = os.path.join(INPUT_FOLDER, mapset)
    if os.path.exists(mapset_dir):
        n_files = len(glob.glob(os.path.join(mapset_dir, '*.tif')))
        print(f"✓ {mapset:20s}: {n_files:4d} files")
    else:
        print(f"✗ {mapset:20s}: Directory not found")

print("\n✓ All WaPOR v3 data downloads complete!")

## 5. Reclassify LCC to LUWA

Convert WaPOR Land Cover Classification to Land Use for Water Accounting (LUWA) classes.

In [ ]:
from WA.LCC_to_LUWA import Rasterize_shape_basin, Adjust_GRaND_reservoir, LCC_to_LUWA

print("="*80)
print("RECLASSIFYING LCC TO LUWA")
print("="*80)

# Define paths
LCC_FOLDER = os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-LCC-A')
LCC_fhs = sorted(glob.glob(os.path.join(LCC_FOLDER, '*.tif')))

if len(LCC_fhs) == 0:
    print(f"ERROR: No LCC files found in {LCC_FOLDER}")
else:
    print(f"Found {len(LCC_fhs)} LCC files to process")
    
    # Use first LCC file as template
    WaPOR_LCC = LCC_fhs[0]
    
    # Optional: Adjust reservoirs (set to None if not needed)
    Resrv_to_Lake = None  # GeoTIFF of reservoirs that are actually natural lakes
    Lake_to_Reserv = None  # GeoTIFF of natural lakes that are actually reservoirs

### 5.1 Create Reservoir Raster Map

In [ ]:
print("\nCreating reservoir raster map...")

# Create output directory
Reservoir_dir = os.path.join(MAIN_DIR, 'Reservoir')
os.makedirs(Reservoir_dir, exist_ok=True)

Basin_Reservoir_tif = os.path.join(Reservoir_dir, 'Reservoir_basin.tif')

# Adjust reservoir if custom shapefiles provided
if (Resrv_to_Lake is not None) and (Lake_to_Reserv is not None):
    print("  Adjusting reservoirs with custom classifications...")
    Adjust_GRaND_reservoir(
        Basin_Reservoir_tif,
        WaPOR_LCC,
        GLOBAL_GRAND_RESERVOIR,
        Resrv_to_Lake,
        Lake_to_Reserv
    )
else:
    print("  Using standard GRanD reservoir classification...")
    Rasterize_shape_basin(
        GLOBAL_GRAND_RESERVOIR,
        WaPOR_LCC,
        Basin_Reservoir_tif
    )

print(f"✓ Reservoir map created: {Basin_Reservoir_tif}")

### 5.2 Create Protected Area Raster Map

In [ ]:
print("\nCreating protected area raster map...")

# Create output directory
Protected_dir = os.path.join(MAIN_DIR, 'Protected')
os.makedirs(Protected_dir, exist_ok=True)

ProtectedArea_tif = os.path.join(Protected_dir, 'ProtectedArea_basin.tif')

Rasterize_shape_basin(
    WDPA_SHAPEFILE,
    WaPOR_LCC,
    ProtectedArea_tif
)

print(f"✓ Protected area map created: {ProtectedArea_tif}")

### 5.3 Convert LCC to LUWA

In [ ]:
print("\nReclassifying LCC to LUWA...")

# Create output directory
LUWA_dir = os.path.join(MAIN_DIR, 'data', 'luwa')
os.makedirs(LUWA_dir, exist_ok=True)

# Process each LCC file
for i, fh in enumerate(LCC_fhs, 1):
    print(f"  [{i}/{len(LCC_fhs)}] Processing {os.path.basename(fh)}...")
    LCC_to_LUWA(
        fh,
        LUWA_dir,
        ProtectedArea_tif,
        Basin_Reservoir_tif
    )

n_luwa = len(glob.glob(os.path.join(LUWA_dir, '*.tif')))
print(f"\n✓ LUWA reclassification complete: {n_luwa} files created")

## 6. Create Monthly Interception

Aggregate dekadal interception data to monthly totals.

In [ ]:
print("="*80)
print("AGGREGATING DEKADAL INTERCEPTION TO MONTHLY")
print("="*80)

# Define input and output folders
Dekadal_I_folder = os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-I-D')
Monthly_I_folder = os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-I-M')
os.makedirs(Monthly_I_folder, exist_ok=True)

# Get list of dekadal rasters
input_fhs = sorted(glob.glob(os.path.join(Dekadal_I_folder, '*.tif')))
print(f"Found {len(input_fhs)} dekadal interception files")

if len(input_fhs) == 0:
    print("ERROR: No dekadal interception files found")
else:
    # Get template for georeferencing
    driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(input_fhs[0])
    
    # Get month dates
    month_dates = pd.date_range(START_DATE, END_DATE, freq='MS')
    print(f"Processing {len(month_dates)} months...\n")
    
    # Process each month
    for idx, date in enumerate(month_dates, 1):
        year = date.year
        month = date.month
        
        print(f"  [{idx}/{len(month_dates)}] Processing {year}-{month:02d}...")
        
        # Find all files for this month by parsing filenames
        month_fhs = []
        for in_fh in input_fhs:
            fname = os.path.basename(in_fh)
            # Parse year-month from filename (format: WAPOR.v3_mm-dekad-1_YYYY-MM-D*.tif)
            if f'{year}-{month:02d}' in fname:
                month_fhs.append(in_fh)
        
        if len(month_fhs) == 0:
            print(f"    WARNING: No dekadal files found for {year}-{month:02d}")
            continue
        
        print(f"    Found {len(month_fhs)} dekadal files for this month")
        
        # Sum dekadal values to get monthly total
        SumArray = np.zeros((ysize, xsize), dtype=np.float32)
        for fh in month_fhs:
            Array = gis.OpenAsArray(fh, nan_values=True)
            SumArray = np.nansum([SumArray, Array], axis=0)
        
        # Save monthly file
        out_fh = os.path.join(
            Monthly_I_folder,
            f'I_WAPOR.v3_level{WAPOR_LEVEL}_mm-month-1_monthly_{year:04d}.{month:02d}.tif'
        )
        gis.CreateGeoTiff(out_fh, SumArray, driver, NDV, xsize, ysize, GeoT, Projection)
    
    n_monthly = len(glob.glob(os.path.join(Monthly_I_folder, '*.tif')))
    print(f"\n✓ Monthly interception aggregation complete: {n_monthly} files created")

## 7. Create Monthly Rainy Days

Calculate the number of rainy days per month from daily precipitation data.
A rainy day is defined as a day with precipitation > 0.201 mm.

In [ ]:
print("="*80)
print("CALCULATING MONTHLY RAINY DAYS")
print("="*80)

# Define paths
Data_Path_P = os.path.join(INPUT_FOLDER, 'L1-PCP-E')
Data_Path_RD = os.path.join(INPUT_FOLDER, 'Rainy_Days')
os.makedirs(Data_Path_RD, exist_ok=True)

# Get list of daily precipitation files
daily_files = sorted(glob.glob(os.path.join(Data_Path_P, '*daily*.tif')))
print(f"Found {len(daily_files)} daily precipitation files\n")

if len(daily_files) == 0:
    print("ERROR: No daily precipitation files found")
else:
    # Generate list of months to process
    Dates = pd.date_range(START_DATE, END_DATE, freq='MS')
    
    for idx, Date in enumerate(Dates, 1):
        year = Date.year
        month = Date.month
        daysinmonth = calendar.monthrange(year, month)[1]
        
        print(f"  [{idx}/{len(Dates)}] Processing {year}-{month:02d} ({daysinmonth} days)...")
        
        # Find all files for this month
        # Format: P_WAPOR.v3_mm-day-1_daily_YYYY.MM.DD.tif
        month_files = [f for f in daily_files if f'daily_{year}.{month:02d}.' in f]
        
        # Check if we have all days
        if len(month_files) != daysinmonth:
            print(f"    WARNING: Expected {daysinmonth} files, found {len(month_files)}")
            if len(month_files) == 0:
                continue
        
        # Initialize arrays
        first_file = month_files[0]
        driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(first_file)
        P_Daily = np.zeros([len(month_files), ysize, xsize], dtype=np.float32)
        
        # Load all daily data for the month
        for i, file_path in enumerate(sorted(month_files)):
            Data = gis.OpenAsArray(file_path, nan_values=True)
            Data[Data < 0] = 0  # Remove negative values
            P_Daily[i, :, :] = Data
        
        # Define rainy days (precipitation > 0.201 mm)
        P_Daily_binary = np.where(P_Daily > 0.201, 1, 0)
        
        # Count number of rainy days
        RD_one_month = np.sum(P_Daily_binary, axis=0).astype(np.float32)
        
        # Save output
        Outname = os.path.join(
            Data_Path_RD,
            f'Rainy_Days_NumOfDays_monthly_{year:04d}.{month:02d}.01.tif'
        )
        gis.CreateGeoTiff(Outname, RD_one_month, driver, NDV, xsize, ysize, GeoT, Projection)
    
    n_rainy_days = len(glob.glob(os.path.join(Data_Path_RD, '*.tif')))
    print(f"\n✓ Monthly rainy days calculation complete: {n_rainy_days} files created")

## 8. Final Summary

In [ ]:
print("\n" + "="*80)
print("FINAL PROCESSING SUMMARY")
print("="*80)

summary_data = {
    'Dataset': [],
    'Location': [],
    'Files': []
}

# Check all output folders
check_folders = [
    ('Daily Precipitation', os.path.join(INPUT_FOLDER, 'L1-PCP-E')),
    ('Monthly Precipitation', os.path.join(INPUT_FOLDER, 'L1-PCP-M')),
    (f'Monthly Reference ET (L{WAPOR_LEVEL})', os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-RET-M')),
    (f'Monthly Actual ET (L{WAPOR_LEVEL})', os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-AETI-M')),
    (f'Dekadal Interception (L{WAPOR_LEVEL})', os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-I-D')),
    (f'Monthly Interception (L{WAPOR_LEVEL})', os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-I-M')),
    (f'Land Cover (L{WAPOR_LEVEL})', os.path.join(INPUT_FOLDER, f'L{WAPOR_LEVEL}-LCC-A')),
    ('LUWA', os.path.join(MAIN_DIR, 'data', 'luwa')),
    ('Rainy Days', os.path.join(INPUT_FOLDER, 'Rainy_Days')),
]

for name, folder in check_folders:
    if os.path.exists(folder):
        n_files = len(glob.glob(os.path.join(folder, '*.tif')))
        summary_data['Dataset'].append(name)
        summary_data['Location'].append(folder)
        summary_data['Files'].append(n_files)

# Create summary dataframe
df_summary = pd.DataFrame(summary_data)
print("\n")
print(df_summary.to_string(index=False))

print("\n" + "="*80)
print("✓ ALL PROCESSING COMPLETE!")
print("="*80)
print(f"\nData ready for water accounting analysis!")
print(f"Input folder: {INPUT_FOLDER}")
print(f"Main folder:  {MAIN_DIR}")